In [3]:
# Dependencies: numpy, matplotlib, pandas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import cantera as ct

print('Running Cantera version: ' + ct.__version__)

def ignition_delay(states, species):
    """计算点火延迟时间"""
    i_ign = states(species).Y.argmax()
    return states.t[i_ign], states.T[i_ign]

def run_simulation(T, P, phi):
    """运行单个条件下的模拟"""
    # 创建气体对象并设置状态
    ideal_gas = ct.Solution('~/0/yaml/Burke2012_s9r23.yaml')
    ideal_gas.TP = T, P * 101325.0  # 转换为帕斯卡
    
    # 设置当量比
    ideal_gas.set_equivalence_ratio(phi=phi, fuel='H2',
                                    oxidizer={'O2': 1.0, 'N2': 3.76})
    
    # 创建反应器网络
    r = ct.Reactor(ideal_gas)
    reactor_network = ct.ReactorNet([r])
    time_history_IG = ct.SolutionArray(ideal_gas, extra=['t'])
    
    # 运行模拟
    t = 0
    counter = 1
    estimated_ignition_delay_time = 0.005
    
    while t < estimated_ignition_delay_time:
        t = reactor_network.step()
        if counter % 20 == 0:
            time_history_IG.append(r.thermo.state, t=t)
        counter += 1
    
    # 计算点火延迟
    tau_IG, _ = ignition_delay(time_history_IG, 'oh')
    
    return tau_IG

def main():
    """主函数：循环计算所有条件并保存结果"""
    # 定义参数范围
    T_range = np.arange(1100, 1525, 25)  # 1100-1500K，间隔25K
    P_range = np.arange(0.8, 1.21, 0.1)  # 0.8-1.2atm，间隔0.1atm
    phi_range = np.arange(0.8, 1.21, 0.1)  # 0.8-1.2，间隔0.1
    
    # 准备存储结果的列表
    results = []
    
    total_cases = len(T_range) * len(P_range) * len(phi_range)
    print(f"总计算条件数: {total_cases}")
    
    start_time = time.time()
    case_count = 0
    
    # 三重循环计算所有条件
    for T in T_range:
        for P in P_range:
            for phi in phi_range:
                case_count += 1
                
                print(f"计算进度: {case_count}/{total_cases} "
                      f"(T={T}K, P={P}atm, phi={phi})")
                
                try:
                    tau_IG = run_simulation(T, P, phi)
                    
                    # 保存结果
                    results.append({
                        'Temperature_K': T/1000,
                        'Pressure_atm': P,
                        'Equivalence_Ratio': phi,
                        'Ignition_Delay_s': tau_IG
                    })
                    
                    print(f"  点火延迟: {tau_IG:.3e} s")
                    
                except Exception as e:
                    print(f"  计算失败: {e}")
                    # 失败时保存NaN值
                    results.append({
                        'Temperature_K': T,
                        'Pressure_atm': P,
                        'Equivalence_Ratio': phi,
                        'Ignition_Delay_s': np.nan
                    })
    
    end_time = time.time()
    print(f"\n总计算时间: {end_time - start_time:.2f} 秒")
    
    # 转换为DataFrame并保存为CSV
    df_results = pd.DataFrame(results)
    df_results.to_csv('ignition_delay_results.csv', index=False)
    print("\n结果已保存到 'ignition_delay_results.csv'")
    
    # 显示前几行数据
    print("\n前5行数据预览:")
    print(df_results.head())
    
    # 统计成功计算的数量
    successful = df_results['Ignition_Delay_s'].notna().sum()
    print(f"\n成功计算: {successful}/{total_cases}")
    
    return df_results

if __name__ == "__main__":
    # 运行主程序
    data = main()
    
    # 读取CSV文件的示例（后续分析使用）
    # data_A = pd.read_csv('ignition_delay_results.csv')
    # 假设特征在前，标签在最后一列
    # X = data_A.iloc[:, :-1].values  # 特征（温度、压力、当量比）
    # y = data_A.iloc[:, -1].values    # 标签（点火延迟）

Running Cantera version: 3.1.0
总计算条件数: 425
计算进度: 1/425 (T=1100K, P=0.8atm, phi=0.8)
  点火延迟: 1.431e-04 s
计算进度: 2/425 (T=1100K, P=0.8atm, phi=0.9)
  点火延迟: 1.398e-04 s
计算进度: 3/425 (T=1100K, P=0.8atm, phi=1.0)
  点火延迟: 1.378e-04 s
计算进度: 4/425 (T=1100K, P=0.8atm, phi=1.1)
  点火延迟: 1.370e-04 s
计算进度: 5/425 (T=1100K, P=0.8atm, phi=1.2)
  点火延迟: 1.348e-04 s
计算进度: 6/425 (T=1100K, P=0.9atm, phi=0.8)
  点火延迟: 1.251e-04 s
计算进度: 7/425 (T=1100K, P=0.9atm, phi=0.9)
  点火延迟: 1.239e-04 s
计算进度: 8/425 (T=1100K, P=0.9atm, phi=1.0)
  点火延迟: 1.218e-04 s
计算进度: 9/425 (T=1100K, P=0.9atm, phi=1.1)
  点火延迟: 1.192e-04 s
计算进度: 10/425 (T=1100K, P=0.9atm, phi=1.2)
  点火延迟: 1.194e-04 s
计算进度: 11/425 (T=1100K, P=1.0atm, phi=0.8)
  点火延迟: 1.115e-04 s
计算进度: 12/425 (T=1100K, P=1.0atm, phi=0.9)
  点火延迟: 1.103e-04 s
计算进度: 13/425 (T=1100K, P=1.0atm, phi=1.0)
  点火延迟: 1.092e-04 s
计算进度: 14/425 (T=1100K, P=1.0atm, phi=1.1)
  点火延迟: 1.080e-04 s
计算进度: 15/425 (T=1100K, P=1.0atm, phi=1.2)
  点火延迟: 1.068e-04 s
计算进度: 16/425 (T=1100K, P=1.1atm, phi

In [4]:
# Dependencies: numpy, matplotlib, pandas
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import cantera as ct

print('Running Cantera version: ' + ct.__version__)

def ignition_delay(states, species):
    """计算点火延迟时间"""
    i_ign = states(species).Y.argmax()
    return states.t[i_ign], states.T[i_ign]

def run_simulation(T, P, phi):
    """运行单个条件下的模拟"""
    # 创建气体对象并设置状态
    ideal_gas = ct.Solution('~/0/yaml/Burke2012_s9r23.yaml')
    ideal_gas.TP = T, P * 101325.0  # 转换为帕斯卡
    
    # 设置当量比
    ideal_gas.set_equivalence_ratio(phi=phi, fuel='H2',
                                    oxidizer={'O2': 2.0, 'N2': 3.76})
    
    # 创建反应器网络
    r = ct.Reactor(ideal_gas)
    reactor_network = ct.ReactorNet([r])
    time_history_IG = ct.SolutionArray(ideal_gas, extra=['t'])
    
    # 运行模拟
    t = 0
    counter = 1
    estimated_ignition_delay_time = 0.005
    
    while t < estimated_ignition_delay_time:
        t = reactor_network.step()
        if counter % 20 == 0:
            time_history_IG.append(r.thermo.state, t=t)
        counter += 1
    
    # 计算点火延迟
    tau_IG, _ = ignition_delay(time_history_IG, 'oh')
    
    return tau_IG

def main():
    """主函数：循环计算所有条件并保存结果"""
    # 定义参数范围
    T_range = np.arange(1100, 1525, 25)  # 1100-1500K，间隔25K
    P_range = np.arange(0.8, 1.21, 0.1)  # 0.8-1.2atm，间隔0.1atm
    phi_range = np.arange(0.8, 1.21, 0.1)  # 0.8-1.2，间隔0.1
    
    # 准备存储结果的列表
    results = []
    
    total_cases = len(T_range) * len(P_range) * len(phi_range)
    print(f"总计算条件数: {total_cases}")
    
    start_time = time.time()
    case_count = 0
    
    # 三重循环计算所有条件
    for T in T_range:
        for P in P_range:
            for phi in phi_range:
                case_count += 1
                
                print(f"计算进度: {case_count}/{total_cases} "
                      f"(T={T}K, P={P}atm, phi={phi})")
                
                try:
                    tau_IG = run_simulation(T, P, phi)
                    
                    # 保存结果
                    results.append({
                        'Temperature_K': T/1000,
                        'Pressure_atm': P,
                        'Equivalence_Ratio': phi,
                        'Ignition_Delay_s': tau_IG
                    })
                    
                    print(f"  点火延迟: {tau_IG:.3e} s")
                    
                except Exception as e:
                    print(f"  计算失败: {e}")
                    # 失败时保存NaN值
                    results.append({
                        'Temperature_K': T,
                        'Pressure_atm': P,
                        'Equivalence_Ratio': phi,
                        'Ignition_Delay_s': np.nan
                    })
    
    end_time = time.time()
    print(f"\n总计算时间: {end_time - start_time:.2f} 秒")
    
    # 转换为DataFrame并保存为CSV
    df_results = pd.DataFrame(results)
    df_results.to_csv('ignition_delay_results_pure.csv', index=False)
    print("\n结果已保存到 'ignition_delay_results_pure.csv'")
    
    # 显示前几行数据
    print("\n前5行数据预览:")
    print(df_results.head())
    
    # 统计成功计算的数量
    successful = df_results['Ignition_Delay_s'].notna().sum()
    print(f"\n成功计算: {successful}/{total_cases}")
    
    return df_results

if __name__ == "__main__":
    # 运行主程序
    data = main()
    

Running Cantera version: 3.1.0
总计算条件数: 425
计算进度: 1/425 (T=1100K, P=0.8atm, phi=0.8)
  点火延迟: 9.841e-05 s
计算进度: 2/425 (T=1100K, P=0.8atm, phi=0.9)
  点火延迟: 9.981e-05 s
计算进度: 3/425 (T=1100K, P=0.8atm, phi=1.0)
  点火延迟: 9.777e-05 s
计算进度: 4/425 (T=1100K, P=0.8atm, phi=1.1)
  点火延迟: 9.670e-05 s
计算进度: 5/425 (T=1100K, P=0.8atm, phi=1.2)
  点火延迟: 9.629e-05 s
计算进度: 6/425 (T=1100K, P=0.9atm, phi=0.8)
  点火延迟: 8.806e-05 s
计算进度: 7/425 (T=1100K, P=0.9atm, phi=0.9)
  点火延迟: 8.742e-05 s
计算进度: 8/425 (T=1100K, P=0.9atm, phi=1.0)
  点火延迟: 8.700e-05 s
计算进度: 9/425 (T=1100K, P=0.9atm, phi=1.1)
  点火延迟: 8.505e-05 s
计算进度: 10/425 (T=1100K, P=0.9atm, phi=1.2)
  点火延迟: 8.623e-05 s
计算进度: 11/425 (T=1100K, P=1.0atm, phi=0.8)
  点火延迟: 7.865e-05 s
计算进度: 12/425 (T=1100K, P=1.0atm, phi=0.9)
  点火延迟: 7.873e-05 s
计算进度: 13/425 (T=1100K, P=1.0atm, phi=1.0)
  点火延迟: 7.824e-05 s
计算进度: 14/425 (T=1100K, P=1.0atm, phi=1.1)
  点火延迟: 7.764e-05 s
计算进度: 15/425 (T=1100K, P=1.0atm, phi=1.2)
  点火延迟: 7.718e-05 s
计算进度: 16/425 (T=1100K, P=1.1atm, phi